## Getting Started with PyTorch on Cloud TPUs

This notebook will show you how to:

* Install PyTorch/XLA on Colab, which lets you use PyTorch with TPUs.
* Run basic PyTorch functions on TPUs, like creating and adding tensors.
* Run PyTorch modules and autograd on TPUs.
* Run PyTorch networks on TPUs.

PyTorch/XLA is a package that lets PyTorch connect to Cloud TPUs and use TPU cores as devices. Colab provides a free Cloud TPU system (a remote CPU host + four TPU chips with two cores each) and installing PyTorch/XLA only takes a couple minutes.

To use PyTorch on Cloud TPUs in your own Colab notebook you can copy this one, or copy the setup cell below and configure your Colab environment to use TPUs.




<h3>  &nbsp;&nbsp;Use Colab Cloud TPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

* On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
* The cell below makes sure you have access to a TPU on Colab.


## Creating and Manipulating Tensors on TPUs

PyTorch uses Cloud TPUs just like it uses CPU or CUDA devices, as the next few cells will show. Each core of a Cloud TPU is treated as a different PyTorch  device.




In [2]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm

As mentioned above, the PyTorch/XLA package (torch_xla) lets PyTorch use TPU devices. The `xla_device()` function returns the TPU's "default" core as a device. This lets PyTorch creates tensors on TPUs:

In [3]:
# Creates a random tensor on xla:1 (a Cloud TPU core)
dev = xm.xla_device()
t1 = torch.ones(3, 3, device = dev)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:0')


See the documentation at http://pytorch.org/xla/ for a description of all public PyTorch/XLA functions. Here `xm.xla_device()` acquired the first Cloud TPU core ('xla:1'). Other cores can be directly acquired, too:

In [4]:
# Creating a tensor on the second Cloud TPU core
second_dev = xm.xla_device(n=2, devkind='TPU')
t2 = torch.zeros(3, 3, device = second_dev)
print(t2)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='xla:2')


/usr/local/lib/python3.11/dist-packages/torch_xla/core/xla_model.py:99: UserWarning: `devkind` argument is deprecated and will be removed in a future release.
  warnings.warn("`devkind` argument is deprecated and will be removed in a "


It is recommended that you use functions like `xm.xla_device()` over directly specifying TPU cores.

Tensors on TPUs can be manipulated like any other PyTorch tensor. The following cell adds, multiplies, and matrix multiplies two tensors on a TPU core:

In [5]:
a = torch.randn(2, 2, device = dev)
b = torch.randn(2, 2, device = dev)
print(a + b)
print(b * 2)
print(torch.matmul(a, b))

tensor([[-2.1709,  0.8687],
        [-0.4169, -2.6102]], device='xla:0')
tensor([[-1.9041,  1.3752],
        [ 0.1211, -1.3700]], device='xla:0')
tensor([[ 1.1726, -0.9614],
        [ 0.3379,  0.9861]], device='xla:0')


This next cell runs a 1D convolution on a TPU core:

In [6]:
# Creates random filters and inputs to a 1D convolution
filters = torch.randn(33, 16, 3, device = dev)
inputs = torch.randn(20, 16, 50, device = dev)
torch.nn.functional.conv1d(inputs, filters)

tensor([[[ -8.3487,  -1.7244,  -0.0633,  ...,   4.4209,   1.7392,  11.0566],
         [  0.4492,  -2.5616,   8.3175,  ...,  -6.2764,   4.4541,   2.6496],
         [  7.5034,   1.0899,   3.5591,  ...,  -2.7368,  -5.9956,  -0.6176],
         ...,
         [ -5.6083,  -2.3524,   5.7621,  ...,  -7.1722,   3.0126,   2.2364],
         [  5.4591,   2.3915,  -2.6300,  ...,   2.5127,  -6.0706,   6.0664],
         [ -6.6130,   0.4147,  13.6904,  ...,  12.4280,   2.1681,   4.4126]],

        [[  3.6752,  -3.8668,   2.8230,  ...,   3.9862,  -8.1474,  12.9051],
         [  9.4716,  -3.6923,  -3.3784,  ...,  -1.3280,  -3.7977,  10.1738],
         [ -3.4809,  -8.7523,  -6.3204,  ..., -10.3281,   6.3623,  10.3201],
         ...,
         [  3.7361,   3.5268,  -1.5678,  ...,   1.1855,  -0.7647,  -5.9642],
         [  4.2686,  -4.6781,   2.3134,  ...,   4.3414,  -5.6396,   7.7923],
         [ -1.2181,   6.3962,   4.0468,  ...,   2.6808,  -2.9050,   5.0465]],

        [[ 18.8890,  -1.9383,   5.9604,  ...

And tensors can be transferred between CPU and TPU. In the following cell, a tensor on the CPU is copied to a TPU core, and then copied back to the CPU again. Note that PyTorch makes copies of tensors when transferring them across devices, so `t_cpu` and `t_cpu_again` are different tensors.



In [7]:
# Creates a tensor on the CPU (device='cpu' is unnecessary and only added for clarity)
t_cpu = torch.randn(2, 2, device='cpu')
print(t_cpu)

t_tpu = t_cpu.to(dev)
print(t_tpu)

t_cpu_again = t_tpu.to('cpu')
print(t_cpu_again)

tensor([[-1.1314, -0.5279],
        [-1.0666, -0.3110]])
tensor([[-1.1314, -0.5279],
        [-1.0666, -0.3110]], device='xla:0')
tensor([[-1.1314, -0.5279],
        [-1.0666, -0.3110]])


## Running PyTorch modules and autograd on TPUs

Modules and autograd are fundamental PyTorch components.

In PyTorch, every stateful function is a module. Modules are Python classes augmented with metadata that lets PyTorch understand how to use them in a neural network. For example, linear layers are modules, as are entire networks. Since modules are stateful, they can be placed on devices, too. PyTorch/XLA lets us place them on TPU cores:


In [8]:
# Creates a linear module
fc = torch.nn.Linear(5, 2, bias=True)

# Copies the module to the XLA device (the first Cloud TPU core)
fc = fc.to(dev)

# Creates a random feature tensor
features = torch.randn(3, 5, device=dev, requires_grad=True)

# Runs and prints the module
output = fc(features)
print(output)

tensor([[ 0.0295, -0.1017],
        [-0.3432,  0.0148],
        [ 0.9068,  0.6823]], device='xla:0', grad_fn=<AddmmBackward0>)


Autograd is the system PyTorch uses to populate the gradients of weights in a neural network. See [here](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py) for details about PyTorch's autograd. When a module is run on a TPU core, its gradients are also populated on the same TPU core by autograd. The following cell demonstrates this:

In [9]:
output.backward(torch.ones_like(output))
print(fc.weight.grad)

tensor([[0.5913, 0.1172, 0.3398, 0.9641, 0.6719],
        [0.5913, 0.1172, 0.3398, 0.9641, 0.6719]], device='xla:0')


## Running PyTorch networks on TPUs

As mentioned above, PyTorch networks are also modules, and so they're run in the same way. The following cell runs a relatively simple PyTorch network from the [PyTorch tutorial docs](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py) on a TPU core:

In [10]:
import torch.nn as nn
import torch.nn.functional as F

# Simple example network from
# https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


# Places network on the default TPU core
net = Net().to(dev)

# Creates random input on the default TPU core
input = torch.randn(1, 1, 32, 32, device=dev)

# Runs network
out = net(input)
print(out)

tensor([[-0.0735,  0.0874,  0.0451,  0.0057,  0.0659, -0.0753,  0.0047, -0.0587,
          0.0899,  0.0225]], device='xla:0', grad_fn=<AddmmBackward0>)


As in the previous snippets, running PyTorch on a TPU just requires specifying a TPU core as a device.